In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/gemstone.csv')

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df =df.drop(labels='id', axis=1)

In [5]:
X = df.drop(labels='price', axis=1)
y = df['price']

In [6]:
## Segragating the num and cate variable 
cate_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(exclude='object').columns

In [7]:
## Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [8]:
from sklearn.impute import SimpleImputer ## handle missing values 
from sklearn.preprocessing import StandardScaler , OrdinalEncoder ## scale the data and encode the ordinal variable
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [9]:
## Numerical pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

## Categorical pipeline
cat_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cate_cols)
])

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [11]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
                       columns = preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test),
                      columns=preprocessor.get_feature_names_out())

In [12]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,2.626061,-2.888129,0.400868,2.233112,2.216066,1.856561,-0.130933,-1.525655,-1.314696
1,-0.845291,0.164716,0.922458,-0.915966,-0.908068,-0.890852,-0.130933,0.937159,-0.648656
2,-0.845291,-1.500472,1.965640,-0.843987,-0.899013,-0.963153,-0.130933,0.321455,-0.648656
3,-0.694363,-0.667878,-0.642314,-0.637048,-0.636405,-0.673951,0.874463,0.937159,-1.314696
4,1.548002,-0.482857,1.444049,1.477333,1.455407,1.393839,-0.130933,-1.525655,0.683424
...,...,...,...,...,...,...,...,...,...
129688,-0.629679,-1.500472,1.965640,-0.547074,-0.518684,-0.645031,-1.136330,0.937159,-0.648656
129689,2.410449,0.442247,2.487231,1.918204,1.871959,1.928861,-1.136330,0.321455,-0.648656
129690,0.922727,0.904800,0.400868,0.991476,0.921135,1.046797,-0.130933,-0.294248,0.017384
129691,-1.039342,-0.667878,-0.642314,-1.212879,-1.197843,-1.252354,-1.136330,-0.294248,2.015504


In [13]:
## Model Building
from sklearn.linear_model import LinearRegression , Lasso , Ridge , ElasticNet
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [14]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [15]:
regression.coef_

array([ 6432.59272318,  -133.11853452,   -70.36485019, -1713.18964719,
        -490.48291102,   -68.02812257,    68.36709467,   464.25812278,
         651.94096231])

In [16]:
regression.intercept_

3979.27372333125

In [17]:
import numpy as np
def evaluate_model(true,predicated):
    mae = mean_absolute_error(true,predicated)
    mse = mean_squared_error(true,predicated)
    rmse = np.sqrt(mse)
    r2_square= r2_score(true,predicated)
    return mae,rmse,r2_square

In [21]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [66]:
## Train mulitple model
models ={
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'KNeighbors': KNeighborsRegressor()
}

trained_models_list = []
model_list = []
r2_list = []

for i in range (len(list(models.keys()))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
    mae,rmse,r2_square = evaluate_model(y_test,y_pred)
    
    print(f'{list(models.keys())[i]}')
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print(f'Mean Absolute Error: {mae}')
    print(f'rmse: {rmse}')
    print(f'R2 Square: {r2_square}')
    
    r2_list.append(r2_square)
    
    print('='*30)
    print('\n')
    

LinearRegression
Model Training Performance
Mean Absolute Error: 679.0440665748289
rmse: 1014.8366526683034
R2 Square: 0.9363188785467224


Lasso
Model Training Performance
Mean Absolute Error: 680.3217525409708
rmse: 1014.8509887386555
R2 Square: 0.9363170793537524


Ridge
Model Training Performance
Mean Absolute Error: 679.0792900256663
rmse: 1014.8385361993734
R2 Square: 0.9363186421629027


ElasticNet
Model Training Performance
Mean Absolute Error: 1067.1442310482983
rmse: 1527.1509830445516
R2 Square: 0.8557943200273045


DecisionTree
Model Training Performance
Mean Absolute Error: 428.4901299311208
rmse: 856.9577948211496
R2 Square: 0.9545914868693401


RandomForest
Model Training Performance
Mean Absolute Error: 310.72757907547384
rmse: 610.8133631013004
R2 Square: 0.9769306438322013


GradientBoosting
Model Training Performance
Mean Absolute Error: 332.11892286869573
rmse: 619.9997859086733
R2 Square: 0.9762315153879534


SVR
Model Training Performance
Mean Absolute Error: 854.